In [4]:
import pandas as pd
import numpy as np
import os
# 1. Veri Yükleme
df = pd.read_csv('../data/raw/urban_building_stock_datasets_17042024.csv') 

print(f"Ham Veri Seti Boyutu: {df.shape}")


columns_to_drop = [
    #output sütunları ve data leak oluşturan sütunların silinmesi
    'Heating_Usage',
    'Water_Systems_Energy',
    'Interior_Lighting_Energy',
    'Interior_Equipment_Energy',
    'Total_Heating_Energy',
    'Photovoltaic_Power',
    'Total_Electricity_Energy',
    'Energy_Use_Intensity',
    'Building_Energy_Rating',  
    'Heating_Energy'           
]

# 3. Sütunları Düşür
df_cleaned = df.drop(columns=columns_to_drop, axis=1, errors='ignore')

# 4. Kontrol
print("-" * 30)
print(f"Temizlenmiş Veri Seti Boyutu: {df_cleaned.shape}")
print("-" * 30)
print("Kalan Sütunlar:")
print(df_cleaned.columns.tolist())

# Hedef değişkenimizin (y) hala orada olduğundan emin olalım
if 'Simple_Building_Energy_Rating' in df_cleaned.columns:
    print("\n✅ Hedef değişken (Simple_Building_Energy_Rating) güvende.")
else:
    print("\n❌ DİKKAT: Hedef değişken yanlışlıkla silindi!")

output_folder = '../data/processed'
os.makedirs(output_folder, exist_ok=True)

# Dosyayı kaydet
# index=False ÇOK ÖNEMLİDİR. Yoksa dosyayı tekrar okurken gereksiz bir "Unnamed: 0" sütunu oluşur.
output_path = f'{output_folder}/cleaned_data_step1.csv'
df_cleaned.to_csv(output_path, index=False)

print(f"✅ Temizlenmiş veri başarıyla şuraya kaydedildi: {output_path}")
print(f"Veri Seti Boyutu: {df_cleaned.shape}")

Ham Veri Seti Boyutu: (1048562, 31)
------------------------------
Temizlenmiş Veri Seti Boyutu: (1048562, 22)
------------------------------
Kalan Sütunlar:
['Simple_Building_Energy_Rating', 'Weather_File', 'Building_Type', 'Renewable_Energy_Usage', 'Floor_Insulation_U-Value', 'Door_Insulation_U-Value', 'Roof_Insulation_U-Value', 'Window_Insulation_U-Value', 'Wall_Insulation_U-Value', 'HVAC_Efficiency', 'Domestic_Hot_Water_Usage', 'Building_Orientation', 'Lighting_Density', 'Occupancy_Level', 'Equipment_Density', 'Heating_Setpoint_Temperature', 'Heating_Setback_Temperature', 'Air_Change_Rate', 'Window_to_Wall_Ratio', 'Total_Building_Area', 'Electricity_Primary_Conversion_Factor', 'Heating_Primary_Conversion_Factor']

✅ Hedef değişken (Simple_Building_Energy_Rating) güvende.
✅ Temizlenmiş veri başarıyla şuraya kaydedildi: ../data/processed/cleaned_data_step1.csv
Veri Seti Boyutu: (1048562, 22)


In [5]:
#birinci veri işleme aşamasında sabit değerler alan iki kolonun temizlenmesi
# veri yükleme
df = pd.read_csv('../data/processed/cleaned_data_step1.csv')

print(f"Silme Öncesi Boyut: {df.shape}")

cols_to_drop = [
    'Electricity_Primary_Conversion_Factor',
    'Heating_Primary_Conversion_Factor'
]

df.drop(columns=cols_to_drop, inplace=True, errors='ignore')

print(f"Sabit Sütunlar Silindikten Sonraki Boyut: {df.shape}")

# Kontrol: Başka tek değerli (sabit) sütun kaldı mı?
# nunique() her sütundaki benzersiz veri sayısını verir. 1 olanlar sabittir.
constant_cols_check = [col for col in df.columns if df[col].nunique() <= 1]

if not constant_cols_check:
    print("✅ Başka sabit (constant) sütun kalmadı. Temiz.")
else:
    print(f"⚠️ Dikkat! Şu sütunlar da sabit görünüyor: {constant_cols_check}")
# Kayıt klasörünü garantiye alalım
output_folder = '../data/processed'
os.makedirs(output_folder, exist_ok=True)

# Dosya adını 'step2' yapıyoruz ki ilk hamleyle karışmasın
output_path = f'{output_folder}/cleaned_data_step2.csv'

# index=False diyerek gereksiz satır numarası sütununu engelliyoruz
df.to_csv(output_path, index=False)

print(f"✅ Sabit sütunlardan arındırılmış veri kaydedildi: {output_path}")
print(f"Kaydedilen Veri Boyutu: {df.shape}")

Silme Öncesi Boyut: (1048562, 22)
Sabit Sütunlar Silindikten Sonraki Boyut: (1048562, 20)
✅ Başka sabit (constant) sütun kalmadı. Temiz.
✅ Sabit sütunlardan arındırılmış veri kaydedildi: ../data/processed/cleaned_data_step2.csv
Kaydedilen Veri Boyutu: (1048562, 20)


In [7]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

#Veri yükleme
df = pd.read_csv('../data/processed/cleaned_data_step2.csv')

print(f"Veri Seti Boyutu: {df.shape}")

#Hedef ve özellikleri ayırma
target_col = 'Simple_Building_Energy_Rating'
X = df.drop(target_col, axis=1)
y = df[target_col]

#Veriyi Bölme (Train / Validation / Test)

#Testi ayırma
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

#Train ne validation ayırma
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.25, random_state=42, stratify=y_train_full
)

print("-" * 30)
print(f"Train Seti (Eğitim):     {X_train.shape}")
print(f"Val Seti (Doğrulama):    {X_val.shape}")
print(f"Test Seti (Final Sınav): {X_test.shape}")
print("-" * 30)

#Pipeline Kurulumu (Imputation + Scaling + Encoding)

#Sayısal ve Kategorik sütunları otomatik seç
numeric_features = X.select_dtypes(include=['float64', 'int64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

#Sayısal Pipeline:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

#Kategorik Pipeline:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

#Ana İşleyici (Preprocessor)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

#Dönüştürme İşlemi (Fit & Transform)
print("Veri işleniyor (Imputation, Scaling, Encoding)...")

#Train setine 'fit' yapıyoruz (kuralları öğreniyor), sonra 'transform' (uyguluyor)
X_train_processed = preprocessor.fit_transform(X_train)

#Val ve Test setlerine SADECE 'transform' yapıyoruz (Train'den öğrendiği kuralları uyguluyor)
X_val_processed = preprocessor.transform(X_val)
X_test_processed = preprocessor.transform(X_test)

#Hedef Değişkeni (y) Dönüştürme
target_encoder = OneHotEncoder(sparse_output=False)

y_train_encoded = target_encoder.fit_transform(y_train.values.reshape(-1, 1))
y_val_encoded = target_encoder.transform(y_val.values.reshape(-1, 1))
y_test_encoded = target_encoder.transform(y_test.values.reshape(-1, 1))

#Sınıf isimlerini alıyoruz
class_names = target_encoder.categories_[0]

print("✅ İşlem Başarılı!")
print(f"İşlenmiş Girdi (X Train): {X_train_processed.shape}")
print(f"İşlenmiş Çıktı (y Train): {y_train_encoded.shape}")
print(f"Sınıflar: {class_names}")

Veri Seti Boyutu: (1048562, 20)
------------------------------
Train Seti (Eğitim):     (629136, 19)
Val Seti (Doğrulama):    (209713, 19)
Test Seti (Final Sınav): (209713, 19)
------------------------------
Veri işleniyor (Imputation, Scaling, Encoding)...
✅ İşlem Başarılı!
İşlenmiş Girdi (X Train): (629136, 24)
İşlenmiş Çıktı (y Train): (629136, 7)
Sınıflar: ['A' 'B' 'C' 'D' 'E' 'F' 'G']


In [8]:
#İşlenmiş verileri kaydetme
output_folder = '../data/processed'

#X (Özellikler)
np.save(f'{output_folder}/X_train.npy', X_train_processed)
np.save(f'{output_folder}/X_val.npy', X_val_processed)
np.save(f'{output_folder}/X_test.npy', X_test_processed)

#y (Hedefler)
np.save(f'{output_folder}/y_train.npy', y_train_encoded)
np.save(f'{output_folder}/y_val.npy', y_val_encoded)
np.save(f'{output_folder}/y_test.npy', y_test_encoded)

#Sınıf isimlerini de kaydetme
np.save(f'{output_folder}/class_names.npy', class_names)

print("💾 Tüm işlenmiş veriler ve sınıf isimleri başarıyla kaydedildi.")

💾 Tüm işlenmiş veriler ve sınıf isimleri başarıyla kaydedildi.
